In [ ]:
import torch
import sys
import os
import rasterio

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

from src.ccsds import create_lossless_compressor, decompress
from tests.utils import generate_simple_test_image

# Create test image [Z, Y, X]
#image = torch.randn(10, 64, 64) * 100  # 10 bands, 64x64 pixels
batch_size = 1
num_bands = 5
dynamic_range = 16
#images = [generate_simple_test_image(num_bands=num_bands, dynamic_range=dynamic_range) for _ in range(batch_size)]
#batch = torch.stack(images)
image = generate_simple_test_image(num_bands=num_bands, dynamic_range=dynamic_range)
print(f'Image shape: {image.shape}\n')
print(f'Image Bit: {image.dtype}\n')

In [ ]:
import rasterio
tif_path = '/home/joshua/Documents/phd_university/code/HSI code examples/aviris_hyperspectral_data/19920612_AVIRIS_IndianPine_Site3.tif'

# Open the TIFF file
with rasterio.open(tif_path) as src:
    # Read the entire file into a numpy array
    hsi_data = src.read()

hsi = torch.from_numpy(hsi_data).float()
hsi.shape

In [ ]:
# Create basic lossless compressor
compressor = create_lossless_compressor(num_bands=num_bands, dynamic_range=dynamic_range)

In [ ]:
# Compress to get complete compressed representation
compressed_data = compressor(image)
print(f"Compression ratio: {compressed_data['compression_statistics']['compression_ratio']:.2f}:1")
print(f"Compressed size: {len(compressed_data['compressed_bitstream'])} bytes")

In [ ]:
# Decompress to reconstruct image
reconstructed = decompress(compressed_data)
print(f"Lossless: max error = {torch.max(torch.abs(image - reconstructed))}")

In [ ]:
from src.optimized import create_optimized_block_adaptive_lossless_compressor

optimized_compressor = create_optimized_block_adaptive_lossless_compressor(
    num_bands=hsi.shape[0],
    optimization_mode='causal',  # Maintains exact causal order
    dynamic_range=dynamic_range,
)

optimized_compressor.device

In [ ]:
# For strict CCSDS-123.0-B-2 compliance
from src.optimized import create_optimized_lossless_compressor

optimized_compressor = create_optimized_lossless_compressor(
    num_bands=hsi.shape[0],
    optimization_mode='causal',  # Maintains exact causal order
    dynamic_range=dynamic_range,
)

optimized_compressor.device

In [ ]:
results = optimized_compressor.compress(hsi)

In [ ]:
results.keys()

In [ ]:
print(results['compressed_size'])
print(results['compression_ratio'])
print(results['mapped_indices'].shape)
for key, stat in results['entropy_stats'].items():
    print(f'{key}: {stat}')

In [ ]:
print(results['reconstructed_samples'].shape)

In [ ]:
from src.ccsds import calculate_psnr

In [ ]:
print(calculate_psnr(hsi.to('cpu'), results['reconstructed_samples'].to('cpu'), 16))

In [ ]:
compressed_data = optimized_compressor.compress(hsi)

In [ ]:
compressed_data